In [2]:
import os, sys
import pyspark

# os.environ["PYSPARK_PYTHON"]="/usr/bin/python3"          
# os.environ["PYSPARK_DRIVER_PYTHON"]="/usr/bin/python3"   
spark = pyspark.sql.SparkSession\
    .builder\
    .master("local")\
    .appName("myApp")\
    .getOrCreate() #Singleton Create

23/10/22 06:27:09 WARN Utils: Your hostname, sojaehwiui-MacBookPro.local resolves to a loopback address: 127.0.0.1; using 172.30.1.29 instead (on interface en0)
23/10/22 06:27:09 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/10/22 06:27:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/10/22 06:27:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### CSV Load

In [3]:
rdd = spark.sparkContext\
    .textFile(os.path.join("data","경기도 의정부시_인구현황_20230831.csv"), use_unicode=True)



In [12]:
# ,를 기준으로 데이터를 쪼갬
plitRdd = rdd.map(lambda x : x.split(',')) 
splitRdd


PythonRDD[8] at RDD at PythonRDD.scala:53

In [8]:
# 헤더 및 이상원소를 제외하기 위해 x[1]이 정수형태인지 판단하고, 그렇지 않으면 제외시킴
def is_integer(x):
    try:
        int(x[1])
        return True
    except ValueError:
        return False
    
deleteFirstRowRdd = splitRdd.filter(is_integer)
deleteFirstRowRdd.collect()


[['������1��',
  '37557 ',
  '19039 ',
  '18518 ',
  '8.08',
  '4.1',
  '3.98',
  '102.81',
  '22514 ',
  '1.67',
  '��\u2d75 �����ν�û',
  '�ο����ǰ�',
  '031-828-2466',
  '2023-06-30'],
 ['������2��',
  '29729 ',
  '14817 ',
  '14912 ',
  '6.4',
  '3.19',
  '3.21',
  '99.36',
  '16007 ',
  '1.86',
  '��\u2d75 �����ν�û',
  '�ο����ǰ�',
  '031-828-2466',
  '2023-06-30'],
 ['ȣ��1��',
  '34658 ',
  '16762 ',
  '17896 ',
  '7.46',
  '3.61',
  '3.85',
  '93.66',
  '15296 ',
  '2.27',
  '��\u2d75 �����ν�û',
  '�ο����ǰ�',
  '031-828-2466',
  '2023-06-30'],
 ['ȣ��2��',
  '33400 ',
  '16088 ',
  '17312 ',
  '7.19',
  '3.46',
  '3.72',
  '92.93',
  '13422 ',
  '2.49',
  '��\u2d75 �����ν�û',
  '�ο����ǰ�',
  '031-828-2466',
  '2023-06-30'],
 ['��ϵ�',
  '19197 ',
  '9156 ',
  '10041 ',
  '4.13',
  '1.97',
  '2.16',
  '91.19',
  '8324 ',
  '2.31',
  '��\u2d75 �����ν�û',
  '�ο����ǰ�',
  '031-828-2466',
  '2023-06-30'],
 ['�Ű�1��',
  '39848 ',
  '19475 ',
  '20373 ',
  '8.57',
  '4.19',
  '4.38',
  '95.

#  (1) 데이터는 한 줄씩 구성되어 있는데, 이 중에 컬럼2만을 추출해서 출력 (아래와 같은 출력이다)

In [10]:
getSecColumn = deleteFirstRowRdd.map(lambda x: x[1].strip())
getSecColumn.collect()

['37557',
 '29729',
 '34658',
 '33400',
 '19197',
 '39848',
 '45593',
 '56595',
 '32054',
 '45618',
 '26064',
 '24476',
 '19543',
 '20446']

# (2) 컬럼2를 map-reduce 합계해서 출력 (헤더를 제외하고, 문자-->숫자로 형변환해야 합산이 된다).

In [11]:
resultRdd = getSecColumn.map(lambda x: int(x)) #형 변환


result = resultRdd.reduce(lambda tot, x: tot + x) #합산
result

464778